<데이터 로드>

In [37]:
import numpy as np
import pandas as pd

df = pd.read_csv('titanic_train.csv')
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


<데이터 전처리>

In [38]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

* null 처리

In [39]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [40]:
df['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [41]:
df['Age'].fillna(df["Age"].median(), inplace=True)
df['Embarked'].fillna('S', inplace=True)

In [42]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

In [43]:
df.drop(['PassengerId', 'Name', 'SibSp', 'Parch', 'Ticket', 'Cabin'], axis=1, inplace=True)
df

,Survived,Pclass,Sex,Age,Fare,Embarked
0,0,3,male,22.0,7.2500,S
1,1,1,female,38.0,71.2833,C
2,1,3,female,26.0,7.9250,S
3,1,1,female,35.0,53.1000,S
4,0,3,male,35.0,8.0500,S
...,...,...,...,...,...,...
886,0,2,male,27.0,13.0000,S
887,1,1,female,19.0,30.0000,S
888,0,3,female,28.0,23.4500,S
889,1,1,male,26.0,30.0000,C


In [44]:
df = pd.get_dummies(df, columns=['Pclass', 'Sex', 'Embarked'])
df

,Survived,Age,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,22.0,7.2500,0,0,1,0,1,0,0,1
1,1,38.0,71.2833,1,0,0,1,0,1,0,0
2,1,26.0,7.9250,0,0,1,1,0,0,0,1
3,1,35.0,53.1000,1,0,0,1,0,0,0,1
4,0,35.0,8.0500,0,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
886,0,27.0,13.0000,0,1,0,0,1,0,0,1
887,1,19.0,30.0000,1,0,0,1,0,0,0,1
888,0,28.0,23.4500,0,0,1,1,0,0,0,1
889,1,26.0,30.0000,1,0,0,0,1,1,0,0


In [45]:
x = torch.FloatTensor(df[['Age', 'Fare', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Embarked_C', 'Embarked_Q', 'Embarked_S']].values)
x.shape

torch.Size([891, 9])

In [46]:
# 정규화
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(x)
x = scaler.transform(x)  

In [47]:
y_data = df['Survived'].values
y = torch.FloatTensor(y_data)
y = y.unsqueeze(1)
y.shape

torch.Size([891, 1])

In [48]:
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=1/5, random_state=0)

X_train = torch.Tensor(X_train)
X_test = torch.Tensor(X_test)
y_train = torch.FloatTensor(y_train)
y_test = torch.FloatTensor(y_test)

ds_train = TensorDataset(X_train, y_train)
ds_test = TensorDataset(X_test, y_test)

# 미니 배치 
# 검증
# 경사하강법 따로 따로 계산해서 속도 개선
loader_train = DataLoader(ds_train, batch_size=60, shuffle=True)   
loader_test = DataLoader(ds_test, batch_size=60, shuffle=False)   

<모델 생성>

In [53]:
# Sequential() 여러개의 모델을 묶어주는 역할
model = nn.Sequential()

model.add_module('fc1', nn.Linear(9, 100))
model.add_module('relu1', nn.ReLU()) 

model.add_module('fc2', nn.Linear(100, 100))
model.add_module('relu2', nn.ReLU())

model.add_module('fc3', nn.Linear(100, 100))
model.add_module('relu3', nn.ReLU())

model.add_module('fc4', nn.Linear(100, 1))
model.add_module('sig', nn.Sigmoid())

print(model)

Sequential(
  (fc1): Linear(in_features=9, out_features=100, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=100, out_features=100, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=100, out_features=100, bias=True)
  (relu3): ReLU()
  (fc4): Linear(in_features=100, out_features=1, bias=True)
  (sig): Sigmoid()
)


<비용함수, 옵티마이저 생성>

In [54]:
loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

<학습>

In [55]:
model.train()  
for e in range(2001):
    for data, targets in loader_train:

        optimizer.zero_grad()  
        outputs = model(data)  
        loss = loss_fn(outputs, targets)  
        loss.backward()  
        optimizer.step()
    if e%200==0:
        print(e, '/2000', ',', 'cost:', loss.item())

0 /2000 , cost: 0.46714845299720764
200 /2000 , cost: 0.39301395416259766
400 /2000 , cost: 0.33628273010253906
600 /2000 , cost: 0.3381471037864685
800 /2000 , cost: 0.2319844663143158
1000 /2000 , cost: 0.25585392117500305
1200 /2000 , cost: 0.19062520563602448
1400 /2000 , cost: 0.25984081625938416
1600 /2000 , cost: 0.38400474190711975
1800 /2000 , cost: 2.263237237930298
2000 /2000 , cost: 0.17969749867916107


<테스팅>
- 이진분류니까 0.5 이상인 것만 확인하면 되는 것이었다

In [56]:
# 값 1개가 나온다 
model.eval()  
correct = 0

with torch.no_grad():  
    for data, targets in loader_test:

        outputs = model(data)  
        # predicted : 최대값의 위치
        predicted = (outputs.data > 0.5).float()
        correct += predicted.eq(targets.data.view_as(predicted)).sum()  

data_num = len(loader_test.dataset)  
print('\n테스트 데이터에서 예측 정확도: {}/{} ({:.0f}%)\n'.format(correct, data_num, 100. * correct / data_num))


테스트 데이터에서 예측 정확도: 148/179 (83%)



In [57]:
# 이렇게 하면 안됨 이건 다중 분류 때 
# model.eval()  
# correct = 0

# with torch.no_grad():  
#     for data, targets in loader_test:

#         outputs = model(data)  
#         # predicted : 최대값의 위치
#         _, predicted = torch.max(outputs.data, 1) 
#         correct += predicted.eq(targets.data.view_as(predicted)).sum()  

# data_num = len(loader_test.dataset)  
# print('\n테스트 데이터에서 예측 정확도: {}/{} ({:.0f}%)\n'.format(correct, data_num, 100. * correct / data_num))